# Focal mechanism calculation using `SKHASH` file format

**Make P-polarity input file i.e., `pol_consensus.csv` file.**

columns:\
        `event_id,event_id2,station,location,channel,p_polarity,origin_latitude,origin_longitude,origin_depth_km`



**pyrocko marker to skshash pol_concensus for individual event marker file.
This is my old  filing system. I make 1 marker file for one event. Now, i have a master marker file for all events.**

In [34]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyperclip


In [35]:
# Set up directory
project_dir = f"{os.getcwd()}/.."
skhash_dir = f"{project_dir}/code/SKHASH2/SKHASH"  ## SKHASH2 is the most recent version
skhash_example_dir = f"{skhash_dir}/examples"
skhash_mtj_dir = f'{skhash_dir}/examples/maacama_SKHASH_MTJ'

############## CHECK THE FILE NAMES CAREFULLY TO MAKE SURE THEY ARE THE MOST RECENT ONES ############################

all_picks_file = f'{project_dir}/results/phasenet_diting_others/phasenet_files/' + 'ALL_PICKS_POL.csv'

eq_cat_file = f'{project_dir}/data/eq_data/1_eq_catalogs/' + 'usgs_3D_gridsort_eq_above_slab2_area.csv'

inv_file = f'{project_dir}/data/eq_data/3_station_catalogs/' + '00_station_inventory_master.txt'

output_path = f'{skhash_mtj_dir}/IN'


# Run the functions

# New style test

In [45]:
from classes_functions.my_class_funcs import SkhashRunner
skrun = SkhashRunner()

skhash_pol_df_pyroko = skrun.PhaseNet2SKHASH_polarity(all_picks_file, eq_cat_file, manual_AI_commons_only=True, pyrocko_only=True, output_path=None)

# drop rows with empty p_polarity
skhash_pol_df_pyroko = skhash_pol_df_pyroko.dropna(subset=['p_polarity'])

skhash_pol_df_pyroko.to_csv(f"{skhash_mtj_dir}/IN/pol_concensus_pyroko.csv", index=False)


/Users/mdarifulislam/Library/CloudStorage/OneDrive-IndianaUniversity/Research/Github/FM2STRESS/FM2STRESS_project/code/classes_functions/my_class_funcs.py:1079: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  skhash_pol_df = pd.concat([skhash_pol_df, event_df], ignore_index=True)


In [46]:

# make station file
skhash_station_df = skrun.make_SKHASH_station_file(given_inventory=inv_file, #skhash_polarity_file=f"{skhash_mtj_dir}/IN/pol_concensus_pyroko.csv",
    keep_Z_only = True, drop_duplicates = True, output_path = None)

skhash_station_df.to_csv(f"{skhash_mtj_dir}/IN/station_all.csv", index=False)

Using the provided merged inventory file: /Users/mdarifulislam/Library/CloudStorage/OneDrive-IndianaUniversity/Research/Github/FM2STRESS/FM2STRESS_project/code/../data/eq_data/3_station_catalogs/00_station_inventory_master.txt


In [47]:

skhash_reverse_df = skrun.make_skhash_reverse_file(f"{skhash_mtj_dir}/IN/pol_concensus_pyroko.csv", output_path=None) # make sure the polarity file has been created
skhash_reverse_df.to_csv(f"{skhash_mtj_dir}/IN/reverse_pyroko.csv", index=False)

In [52]:
from classes_functions.my_class_funcs import SkhashRunner
%reload_ext autoreload
%autoreload 2

skrun = SkhashRunner()
# control_file = skrun.edit_skhash_control_file(
#     conpfile = f'{skhash_mtj_dir}/IN/pol_consensus.csv',
#     stfile = f'{skhash_mtj_dir}/IN/station.csv',
#     outfile1 = f'{skhash_mtj_dir}/OUT/out.csv',
#     outfile2 = f'{skhash_mtj_dir}/OUT/out2.csv',
#     outfile_pol_agree = f'{skhash_mtj_dir}/OUT/out_polagree.csv',
#     outfile_pol_info = f'{skhash_mtj_dir}/OUT/out_polinfo.csv',
#     vmodel_paths = f'{skhash_example_dir}/velocity_models_MTJ/vz.MTJ.txt',
# )

control_file = skrun.edit_skhash_control_file(
    conpfile = 'examples/maacama_SKHASH_MTJ/IN/pol_concensus_pyroko.csv',
    stfile = 'examples/maacama_SKHASH_MTJ/IN/station_all.csv',
    outfile1 = 'examples/maacama_SKHASH_MTJ/OUT/out_pyrocko.csv',
    outfile2 = 'examples/maacama_SKHASH_MTJ/OUT/out2_pyrocko.csv',
    outfile_pol_agree = 'examples/maacama_SKHASH_MTJ/OUT/out_polagree_pyrocko.csv',
    outfile_pol_info = 'examples/maacama_SKHASH_MTJ/OUT/out_polinfo_pyrocko.csv',
    outfolder_plots = 'examples/maacama_SKHASH_MTJ/OUT/plots',
    plot_station_names = True,
    plot_acceptable_solutions = True,
)

In [53]:
# write the control file into a text file   
with open(f'{skhash_mtj_dir}/control_test.txt', 'w') as f:
    f.write(control_file)

# Run SkHash in bash
cd into SkHash folder and run the following code in terminal

In [54]:
bash_script = f"""#!/bin/bash
cd {skhash_dir}

# Activate conda pythoon environment named 'phasenet' [********change miniconda3 to anaconda3 if using Anaconda]
source ~/miniconda3/etc/profile.d/conda.sh # or source ~/anaconda3/etc/profile.d/conda.sh in case of Anaconda
# conda init
conda activate obspy

python3 SKHASH.py examples/maacama_SKHASH_MTJ/control_test.txt
"""

with open(f'{skhash_dir}/run_skhash.sh', 'w') as f:
    f.write(bash_script)

# Make the script executable
print("Making the script executable...")
os.system(f"chmod +x {skhash_dir}/run_skhash.sh")

Making the script executable...


0

In [55]:
# Run the script
# run = input("Do you want to run PhaseNet now? (y/n): ")
run = "y"
if run.lower() == "y":
    print("Running SKHASH...")
    os.system(f"{skhash_dir}/run_skhash.sh")
else:
    print("SKHASH will not be run. Exiting...")

Running SKHASH...
SKHASH v0.1 (2024-02-16)
Control file: examples/maacama_SKHASH_MTJ/control_test.txt
Discarded 65 of the 1785 polarity measurements with source-receiver distances >200 km
Creating lookup table (0/0): examples/velocity_models_MTJ/vz.MTJ.txt
	Created table.
Computing mechanisms in parallel...
*WARNING: Perturbed locations for event_id 'nc40234034' are deeper than lookup table depth range (39). Setting these perturbed depths to 39 km.
7 / 46	(nc51208596)
	Maximum azimuthal gap (235.29) > max_agap (190). Skipping.
*sdr_from_vector warning, horz fault, strike undefined
*sdr_from_vector warning, horz fault, strike undefined
*sdr_from_vector warning, horz fault, strike undefined
0 / 46	(nc40216664)
	S: 131.5581   D: 52.8695   R: -113.4052   U: 20.4576   Q: B
	Runtime: 3.44 sec
6 / 46	(nc51207829)
	S: 128.2628   D: 79.7394   R: -170.1754   U: 14.7956   Q: C
	Runtime: 3.37 sec
*WARNING: Perturbed locations for event_id 'nc71180461' are shallower than lookup table depth range (0